In [1]:
import pandas as pd
import json

In [2]:
df_final = pd.read_csv("C:/Users/Usuario/Documents/Adalab49/Módulo 3/Proyecto_módulo 3/Copia/project-da-promo-49-modulo-3-team-5/fase_2_transformacion/df_final.csv")


In [3]:
df_final.head()

,Unnamed: 0,employee_id,age,attrition,businesstravel,$_dailyrate,department,distancefromhome,education,educationfield,...,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,datebirth,$_salary,remotework
0,0,1,51,No,non_travel,2015.72,NaN,6,3,NaN,...,30.037232,5,3.000000,20,NaN,15,15,1972,195370.000000,yes
1,1,2,52,No,non_travel,2063.39,NaN,1,4,life sciences,...,34.000000,5,3.000000,33,NaN,11,9,1971,199990.000000,yes
2,2,3,42,No,travel_rarely,1984.25,research & development,4,2,technical degree,...,22.000000,3,2.802326,22,NaN,11,15,1981,192320.000000,yes
3,3,4,47,No,travel_rarely,1771.40,NaN,2,4,medical,...,26.670674,2,2.791187,20,NaN,5,6,1976,171690.000000,no
4,4,5,46,No,non_travel,1582.77,NaN,3,3,technical degree,...,11.329230,5,3.000000,19,NaN,2,8,1977,80223.043381,no


In [4]:
df_final.columns

Index(['Unnamed: 0', 'employee_id', 'age', 'attrition', 'businesstravel',
       '$_dailyrate', 'department', 'distancefromhome', 'education',
       'educationfield', 'environmentsatisfaction', 'gender', '$_hourlyrate',
       'jobinvolvement', 'joblevel', 'jobrole', 'jobsatisfaction',
       'maritalstatus', '$_monthlyincome', '$_monthlyrate',
       'numcompaniesworked', 'overtime', '%_percentsalaryhike',
       'performancerating', 'relationshipsatisfaction', 'standardhours',
       'stockoptionlevel', 'totalworkingyears', 'trainingtimeslastyear',
       'worklifebalance', 'yearsatcompany', 'yearsincurrentrole',
       'yearssincelastpromotion', 'yearswithcurrmanager', 'datebirth',
       '$_salary', 'remotework'],
      dtype='object')

In [5]:
df_final.columns = df_final.columns.str.strip()
#Para borrar espacios en blanco

In [6]:
df_final = df_final.loc[:, ~df_final.columns.str.contains("^Unnamed")]
# Para borrar las columnas Unnamed

In [7]:
df_final.fillna(value='', inplace=True) # Esto reemplaza NaN por None, que se convierte en NULL en MySQL.

C:\Users\Usuario\AppData\Local\Temp\ipykernel_23292\1231838440.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.fillna(value='', inplace=True) # Esto reemplaza NaN por None, que se convierte en NULL en MySQL.


In [8]:
# Hay que dividirlo en 3 tablas porque ahora solo tenemos una que queremos convertir en 3
# Tabla 1: employee_details
df_employee_details = df_final[['employee_id', 'age', 'distancefromhome', 'education',
                                'educationfield', 'gender', 'maritalstatus', 'numcompaniesworked',
                                'totalworkingyears', 'datebirth']]

In [9]:
# Tabla 2: employee_satisfaction
df_employee_satisfaction = df_final[['employee_id', 'environmentsatisfaction', 'jobsatisfaction',
                                     'relationshipsatisfaction', 'worklifebalance', 'jobinvolvement']]

In [10]:
# Tabla 3: job_details
df_job_details = df_final[['employee_id', 'attrition', 'businesstravel', '$_dailyrate', 'department',
                           '$_hourlyrate', 'joblevel', 'jobrole', '$_monthlyincome', '$_monthlyrate',
                           'overtime', '%_percentsalaryhike', 'performancerating', 'standardhours',
                           'stockoptionlevel', 'trainingtimeslastyear', 'yearsatcompany',
                           'yearsincurrentrole', 'yearssincelastpromotion', 'yearswithcurrmanager',
                           '$_salary', 'remotework']]


In [11]:
df_job_details.columns = df_job_details.columns.str.replace(r'[$%]', '', regex=True)
df_job_details.columns = df_job_details.columns.str.replace(r'^_', '', regex=True)
# Eliminar símbolos que dan error

In [12]:
lista_details = df_employee_details.values.tolist()
lista_satisfaction = df_employee_satisfaction.values.tolist()
lista_job = df_job_details.values.tolist()

In [13]:
print(df_job_details.columns.tolist())  


['employee_id', 'attrition', 'businesstravel', 'dailyrate', 'department', 'hourlyrate', 'joblevel', 'jobrole', 'monthlyincome', 'monthlyrate', 'overtime', 'percentsalaryhike', 'performancerating', 'standardhours', 'stockoptionlevel', 'trainingtimeslastyear', 'yearsatcompany', 'yearsincurrentrole', 'yearssincelastpromotion', 'yearswithcurrmanager', 'salary', 'remotework']


In [14]:
import mysql.connector
from mysql.connector import errorcode

In [15]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database ='proyecto_abc')

mycursor = cnx.cursor() # creamos el "camioncito que transporta"

In [16]:
# Consulta SQL para INTRODUCIR DATOS de employee_details
query = """
INSERT INTO employee_details (employee_id,age,distancefromhome,education,educationfield,gender,maritalstatus,numcompaniesworked,totalworkingyears,datebirth)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""
mycursor.executemany(query, lista_details)

In [17]:
# Consulta SQL para INTRODUCIR DATOS de employee_satisfaction

query2 = """
INSERT INTO employee_satisfaction (employee_id,environmentsatisfaction,jobsatisfaction,relationshipsatisfaction,worklifebalance,jobinvolvement)
VALUES (%s,%s,%s,%s,%s,%s)
"""
mycursor.executemany(query2, lista_satisfaction)

In [18]:
# Consulta SQL para INTRODUCIR DATOS de job_details
query3 = """
INSERT INTO job_details (employee_id, attrition, businesstravel, dailyrate, department, hourlyrate, joblevel, jobrole, monthlyincome, monthlyrate, overtime, percentsalaryhike, performancerating, standardhours, stockoptionlevel, trainingtimeslastyear, yearsatcompany, yearsincurrentrole, yearssincelastpromotion, yearswithcurrmanager, salary, remotework)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

# Asegúrate de que lista_job tiene la misma cantidad de columnas que la consulta
mycursor.executemany(query3, lista_job)

In [20]:
lista_de_tuplas = []
# Insertar datos
try:
    mycursor.executemany(query, lista_de_tuplas)
    cnx.commit()
    print("✅ Datos insertados correctamente.")
    print(mycursor.rowcount, "registros insertados.")
except mysql.connector.Error as err:
    print("❌ Error al insertar:", err)
    cnx.rollback()

# Cerrar conexión
mycursor.close()
cnx.close()

✅ Datos insertados correctamente.
1614 registros insertados.
